# Podcasts

#### Imports

In [1]:
import queue

import numpy as np
import sounddevice as sd

from openai import OpenAI

#### Teaser Setup

In [2]:
tool_response = {
    "topic": "Sun", "teaser": "Get ready to blast off on an adventure to the center of our solar system, where a giant ball of fire and energy called the Sun reigns supreme - find out why this star is so important for life on Earth and what makes it shine so bright! Tune in to discover the secrets of the Sun and unlock the mysteries of our universe!", "llm": "gpt-4o-mini", "queries": ["How hot is it on the sun?"]
}

#### Text-to-Speech Full Response

In [3]:
client = OpenAI()

# Sampling rate and block size
sample_rate = 24_000  # Matches OpenAI TTS sample rate
block_size = 1024

# Create streaming response from OpenAI API
with client.audio.speech.with_streaming_response.create(
        model = 'tts-1',
        voice = 'ash',
        input = tool_response['teaser'],
        response_format = 'pcm'
) as response:
    # Create a generator that yields PCM audio chunks
    def audio_generator():
        for chunk in response.iter_bytes(block_size):
            yield np.frombuffer(chunk, dtype = np.int16) / 32768.0  # Normalize PCM to -1.0 to 1.0

    # Play audio using sounddevice
    sd.play(np.concatenate(list(audio_generator())), samplerate = sample_rate)
    sd.wait()  # Ensure playback completes before script exits

#### Text-to-Speech Real-Time Streaming

In [ ]:
### NOT YET IMPLEMENTED ###

client = OpenAI()

# Sampling rate
sample_rate = 24_000  # Matches OpenAI TTS sample rate
block_size = 1024

# Create a queue to hold audio chunks
audio_queue = queue.Queue()
leftover_buffer = np.array([], dtype = np.float32)  # Buffer for leftover audio samples

# Callback function to play streaming audio
def callback(outdata, frames, time, status):
    global leftover_buffer
    
    if status:
        print(status)

    # If buffer is empty, try getting more audio data
    while len(leftover_buffer) < frames:
        try:
            new_chunk = audio_queue.get_nowait()
            leftover_buffer = np.concatenate((leftover_buffer, new_chunk))
        except queue.Empty:
            break  # No more data available

    # Fill the requested number of frames
    if len(leftover_buffer) >= frames:
        outdata[:] = leftover_buffer[:frames].reshape(-1, 1)
        leftover_buffer = leftover_buffer[frames:]  # Remove used frames
    else:
        # If not enough data, pad with silence
        outdata[:len(leftover_buffer)] = leftover_buffer.reshape(-1, 1)
        outdata[len(leftover_buffer):] = 0
        leftover_buffer = np.array([], dtype = np.float32)  # Reset buffer

# Open a non-blocking output audio stream
with sd.OutputStream(samplerate = sample_rate, channels = 1, dtype = 'float32', callback = callback):

    # Create streaming response from OpenAI API
    with client.audio.speech.with_streaming_response.create(
            model = 'tts-1',
            voice = 'ash',
            input = tool_response['teaser'],
            response_format = 'pcm'
    ) as response:
        
        for chunk in response.iter_bytes(block_size):
            # Convert PCM bytes to NumPy float32 array
            audio_chunk = np.frombuffer(chunk, dtype = np.int16) / 32768.0
            audio_queue.put(audio_chunk)

        # Wait until all audio is played
        audio_queue.join()